# RAG의 핵심 - Retrieval (정보 검색)

LLM이 학습하지 않은 외부 데이터를 참고하여 답변하도록 만드는 **RAG(Retrieval-Augmented Generation)**의 핵심, **Retrieval** 과정을 실습합니다.

**학습 목표:**
1. **Document Loader:** 웹페이지, PDF, 텍스트 파일 등 외부 데이터를 LangChain Document 형식으로 불러오기
2. **Text Splitter:** 긴 문서를 LLM이 처리하기 좋은 크기의 청크(Chunk)로 나누기
3. **Embedding & Vector Store:** 텍스트를 의미 기반 벡터로 변환하고 검색 가능한 저장소에 저장하기
4. **Retriever:** 사용자의 질문과 관련된 최적의 문서를 검색하여 LLM에 전달하기

### 1. 환경 설정 (Environment Setup)

필요한 패키지를 설치하고 환경 변수를 로드합니다.
- `tiktoken`: 텍스트 분할 시 토큰 수를 계산하기 위해 필요
- `faiss-cpu`: 벡터 저장소(Vector Store)로 사용
- `pypdf`: PDF 파일 로딩용

In [ ]:
%pip install langchain langchain-community langchain-openai langchain-huggingface wikipedia pypdf tavily-python tiktoken faiss-cpu sentence-transformers -Uqqq

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

# 1. API Key 로드
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")
os.environ["TAVILY_API_KEY"] = os.getenv("tavily_key")

# 2. LangSmith Tracing 설정
os.environ["LANGSMITH_TRACING"] = 'true'
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")

---### 2. Document (문서 객체)

LangChain에서 모든 텍스트 데이터는 `Document`라는 표준 객체로 변환되어 처리됩니다.
두 가지 핵심 속성을 가집니다:
1. `page_content`: 실제 텍스트 내용
2. `metadata`: 출처(source), 제목, 페이지 번호 등 부가 정보

In [ ]:
from langchain_core.documents import Document

# 임의의 텍스트로 Document 객체 생성 예시
doc = Document(
    page_content='LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.',
    metadata={
        'source': 'manual.txt',  # 출처
        'page': 1,               # 페이지 번호
        'author': 'gimdabin'     # 작성자 등 다양한 정보 추가 가능
    }
)

print("--- 본문 ---")
print(doc.page_content)
print("\n--- 메타데이터 ---")
print(doc.metadata)

---### 3. Document Loader (문서 로더)

외부의 다양한 데이터 소스(Web, PDF, CSV 등)를 `Document` 객체 리스트로 불러옵니다.

#### 3-1. WebBaseLoader
특정 URL의 웹페이지 내용을 크롤링하여 가져옵니다. 네이버 뉴스 등의 본문을 가져올 때 유용합니다.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

url = "https://n.news.naver.com/mnews/article/005/0001830299"

# 일부 웹사이트는 봇 접근을 차단하므로, 브라우저처럼 보이게 헤더(User-Agent)를 설정합니다.
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

loader = WebBaseLoader(url, header_template=header)
docs = loader.load()

# 결과 확인 (너무 길어서 앞부분 500자만 출력)
print(f"문서 개수: {len(docs)}")
print(f"제목: {docs[0].metadata.get('title')}")
print(f"\n본문(일부):\n{docs[0].page_content[:500]}...")

#### 3-2. PyPDFLoader
PDF 파일을 페이지 단위로 읽어옵니다.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# 현재 폴더에 'The_Adventures_of_Tom_Sawyer.pdf' 파일이 있다고 가정
if os.path.exists('The_Adventures_of_Tom_Sawyer.pdf'):
    loader = PyPDFLoader('The_Adventures_of_Tom_Sawyer.pdf')
    pdf_docs = loader.load()

    print(f"총 페이지 수: {len(pdf_docs)}")
    print(f"15페이지 내용 일부:\n{pdf_docs[14].page_content[:300]}...")
else:
    print("PDF 파일이 없습니다. 실습을 위해 파일을 준비해주세요.")

---### 4. Text Splitter (텍스트 분할)

긴 문서를 통째로 LLM에 넣으면 **토큰 제한(Token Limit)**에 걸리거나, 검색 정확도가 떨어질 수 있습니다.
따라서 문서를 의미 있는 단위(Chunk)로 잘개 쪼개는 과정이 필요합니다.

- `RecursiveCharacterTextSplitter`: 문단 -> 문장 -> 단어 순으로 자연스럽게 자르는 가장 일반적인 분할기

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. 분할기 생성
# chunk_size: 각 청크의 최대 글자 수
# chunk_overlap: 청크 간 겹치는 글자 수 (문맥 안 끊기게)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

# 2. 문서 분할 수행 (PDF 문서 사용)
if 'pdf_docs' in locals():
    split_docs = text_splitter.split_documents(pdf_docs[:5])  # 앞 5페이지만 실습
    print(f"분할 전 문서 수: 5개")
    print(f"분할 후 청크 수: {len(split_docs)}개")
    print(f"\n첫 번째 청크 내용:\n{split_docs[0].page_content}")
else:
    # PDF가 없을 경우 임시 텍스트로 실습
    txt = "LangChain은 아주 유용한 도구입니다. " * 50
    split_docs = text_splitter.create_documents([txt])
    print(f"분할 후 청크 수: {len(split_docs)}개")

---### 5. Embedding (임베딩)

텍스트를 컴퓨터가 이해할 수 있는 **숫자 벡터(Vector)**로 변환하는 과정입니다.
유사한 의미를 가진 텍스트는 벡터 공간에서 서로 가깝게 위치하게 됩니다.

- `OpenAIEmbeddings`: 성능이 좋지만 유료 (API 호출)
- `HuggingFaceEmbeddings`: 로컬에서 무료로 실행 가능 (성능 준수)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np

# 1. OpenAI 임베딩 모델 (3-small)
embeddings_openai = OpenAIEmbeddings(model='text-embedding-3-small')

text = "강아지는 귀엽다."
vector = embeddings_openai.embed_query(text)
print(f"OpenAI 벡터 차원수: {len(vector)} (보통 1536차원)")
print(f"앞 5개 값: {vector[:5]}")

# 2. HuggingFace 로컬 임베딩 모델 (sentence-transformers)
# 무료이고 로컬에서 돌아가지만, 처음 다운로드 시 시간이 걸림
# embeddings_hf = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

---### 6. Vector Store & Retriever (벡터 저장소와 검색기)

임베딩된 벡터들을 저장하고, 질문(Query)과 가장 유사한 문서를 찾아주는 검색 엔진입니다.
여기서는 가볍고 빠른 **FAISS**를 사용합니다.

In [ ]:
from langchain_community.vectorstores import FAISS

# 1. 벡터 저장소 생성 (문서들을 벡터화해서 저장)
# split_docs: 위에서 분할한 문서 청크들
# embeddings_openai: 벡터화에 사용할 임베딩 모델
if 'split_docs' in locals():
    vector_db = FAISS.from_documents(split_docs, embeddings_openai)
    print("FAISS 인덱스 생성 완료")

    # 2. 로컬에 저장 (선택)
    vector_db.save_local("./db/faiss_index")

    # 3. 검색 (Similarity Search)
    query = "Tom은 누구입니까?"  # PDF 내용 관련 질문
    results = vector_db.similarity_search(query, k=3)  # 상위 3개 검색

    print(f"\n['{query}'] 검색 결과:")
    for i, res in enumerate(results, 1):
        print(f"{i}. {res.page_content[:100]}...")

### 7. Retriever Interface
VectorStore를 **Retriever** 인터페이스로 변환하면, Chain(`|`)에 바로 연결하여 사용할 수 있습니다.

In [ ]:
# Retriever로 변환
retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 검색 옵션 설정
)

# invoke로 바로 검색 가능
docs = retriever.invoke("Huck Finn은 누구인가?")
print(f"검색된 문서 수: {len(docs)}")

---### 8. RAG Chain 구성해보기 (종합 예제)
검색된 문서를 프롬프트에 넣고 LLM에게 답변하게 만드는 최종 단계입니다.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 프롬프트 정의
prompt = PromptTemplate.from_template("""
당신은 친절한 AI 어시스턴트입니다. 아래 Context를 바탕으로 사용자의 질문에 답하세요.
Context: {context}

Question: {question}
Answer:
""")

# 2. 모델 정의
llm = ChatOpenAI(model="gpt-4o-mini")

# 3. 체인 구성 (LCEL)
# 질문 -> Retriever가 문서 검색 -> context에 주입
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 실행
print(chain.invoke("Tom Sawyer는 어떤 인물인가요?"))